In [1]:
from common import *
from decimal import Decimal
from pathlib import Path

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [ ]:
# dg.y.makesw().shape ==> (258, 3017)

In [2]:
def submit(y_, name):
    '''Params - y_: 1-D `ndarray` of the final actual values to submit, name: `str` of the file name
    Saves `y_` in the given name csv file in data/submit. If `y_` is a `Series`, the index must be the desired index.'''
    if isinstance(y_, pd.DataFrame) and y_.index.name == 'Day':
        assert columns.name == 'Stock'
        y_ = y_.loc[y_.index.isin(set(dg.Day.unique())), dg.y.makesw().columns]
        y_.sort_index(axis=0, inplace=True)
        y_.sort_index(axis=1, inplace=True)
        y_ = y_.makeil_like(dg)
    elif isinstance(y_, pd.DataFrame) or isinstance(y_, pd.Series):
        assert y_.index.name == 'Index'
        y_ = y_.loc[dg.index]
    assert y_.shape == dg.y.shape
    assert y_.dtype == np.float64 or y_.dtype == np.float32
    ypred = (pd.Series(y_).rename('y').astype(str).map(Decimal) / 10000).astype(str)
    ypred.index.name = 'Index'
    path = Path(top_dir + 'data/submit/' + name + '.csv')
    assert not path.is_file(), 'Please manually delete old submission with the same name first'
    ypred.to_csv(path, header=True)

In [ ]:
# generate hdf5 from original files
if __name__ == '__main__':
    from decimal import Decimal
    df = pd.read_csv('train.csv', index_col=0, dtype={'y': str})
    df.index = df.index - N_TRAIN
    df.y = (df.y.map(Decimal) * 10000).astype(float)
    df.columns.values[-1] = 'w'
    df.to_hdf(top_dir + '/data/given/train.hdf5', mode='w', key='df')
    dg = pd.read_csv('test.csv', index_col=0)
    dg.to_hdf(top_dir + '/data/given/test.hdf5', mode='w', key='dg')

In [ ]:
def submission(name):
    sub = pd.read_csv(top_dir + 'data/submit/' + name + '.csv', index_col=0, dtype={'y': str})
    sub.y = (sub.y.map(Decimal) * 10000).astype(float)
    return sub

In [9]:
if __name__ == '__main__':
    zero = np.zeros(len(dg.index))
    submit(zero, 'zero')
    stock_mean = dg.Stock.map(df.groupby('Stock').y.mean()).fillna(0)
    submit(stock_mean, 'stock_mean')